## 1. Carregando os dados dos embeddings

In [1]:
# Função para restaurar embeddings dos documentos de arquivos pickle

import os
import pickle
import numpy as np

###############################################################################
# Modelos
MODELO = 'rufimelo/Legal-BERTimbau-sts-large-ma-v3'
MAX_SEQ_LENGTH = 512

#MODELO = 'sentence-transformers/paraphrase-multilingual-mpnet-base-v2'
#MAX_SEQ_LENGTH = 128
###############################################################################

CAMINHO_MODELO = MODELO.split("/")[-1]

# Pasta com os dados
PASTA_DADOS = './dados/'

# Pasta com os embeddings dos documentos
PASTA_DOCUMENTOS = f'{PASTA_DADOS}outputs/6_gera_embeddings_js/{CAMINHO_MODELO}/'                   

# Pasta com os embeddings das queries
PASTA_QUERIES = f'{PASTA_DADOS}outputs/7_gera_embeddings_termos/{CAMINHO_MODELO}/'

# Pasta com os resultados do caderno
PASTA_RESULTADO_CADERNO = f'{PASTA_DADOS}outputs/8_armazena_dados_em_banco_vetorial/{CAMINHO_MODELO}/'

# Seleciona o tipo de camada oculta
TIPO_CAMADA_OCULTA = 'mean_hidden_state'
#TIPO_CAMADA_OCULTA = 'cls_hidden_state'

# Função para reconstruir dicionário a partir de lotes
def reconstruir_dicionario_a_partir_de_lotes(vetor_de_dicionarios):
    dicionario_reconstruido = {}
    
    # Inicializando listas vazias para cada chave no primeiro dicionário do vetor
    for chave in vetor_de_dicionarios[0].keys():
        dicionario_reconstruido[chave] = []
    
    # Iterando sobre cada dicionário no vetor e concatenando os valores para cada chave
    for dicionario_lote in vetor_de_dicionarios:
        for chave, valores in dicionario_lote.items():
            dicionario_reconstruido[chave].extend(valores)
    
    # Transforma numpy array em tensor
    #dicionario_reconstruido['cls_hidden_state'] = torch.tensor(np.array(dicionario_reconstruido['cls_hidden_state']))
    #dicionario_reconstruido['mean_hidden_state'] = torch.tensor(np.array(dicionario_reconstruido['mean_hidden_state']))
    
    dicionario_reconstruido['cls_hidden_state'] = np.array(dicionario_reconstruido['cls_hidden_state'])
    dicionario_reconstruido['mean_hidden_state'] = np.array(dicionario_reconstruido['mean_hidden_state'])
    
    return dicionario_reconstruido

# Função para restaurar embeddings dos documentos de arquivos pickle
def restaurar_doc_encoded_de_pickle(pasta_resultado_caderno):
    # Lista para armazenar os dicionários lidos dos arquivos .pickle
    doc_encoded_restaurado = []

    # Listando todos os arquivos .pickle no diretório especificado
    arquivos_pickle = [arq for arq in os.listdir(pasta_resultado_caderno) if arq.endswith('.pickle')]

    # Ordenando os arquivos pelo número (assumindo que os nomes dos arquivos seguem o padrão embeddings_js_X.pickle)
    arquivos_pickle.sort(key=lambda x: int(x.split('_')[-1].split('.')[0]))

    # Lendo cada arquivo .pickle e restaurando o dicionário
    for nome_arquivo in arquivos_pickle:
        caminho_arquivo = os.path.join(pasta_resultado_caderno, nome_arquivo)
        with open(caminho_arquivo, 'rb') as arquivo_pickle:
            dicionario_restaurado = pickle.load(arquivo_pickle)
            doc_encoded_restaurado.append(dicionario_restaurado)

    return reconstruir_dicionario_a_partir_de_lotes(doc_encoded_restaurado)

# Função para restaurar embeddings das queries de arquivos pickle
def restaurar_query_encoded_de_pickle(pasta_resultado_caderno):
    # Lista para armazenar os dicionários lidos dos arquivos .pickle
    query_encoded_restaurado = []

    # Listando todos os arquivos .pickle no diretório especificado
    arquivos_pickle = [arq for arq in os.listdir(pasta_resultado_caderno) if arq.endswith('.pickle')]

    # Ordenando os arquivos pelo número (assumindo que os nomes dos arquivos seguem o padrão embeddings_query_X.pickle)
    arquivos_pickle.sort(key=lambda x: int(x.split('_')[-1].split('.')[0]))

    # Lendo cada arquivo .pickle e restaurando o dicionário
    for nome_arquivo in arquivos_pickle:
        caminho_arquivo = os.path.join(pasta_resultado_caderno, nome_arquivo)
        with open(caminho_arquivo, 'rb') as arquivo_pickle:
            dicionario_restaurado = pickle.load(arquivo_pickle)
            query_encoded_restaurado.append(dicionario_restaurado)

    return reconstruir_dicionario_a_partir_de_lotes(query_encoded_restaurado)

In [2]:
# Carrega dados de queries
query_encoded_restaurado = restaurar_query_encoded_de_pickle(PASTA_QUERIES)
xq = query_encoded_restaurado[TIPO_CAMADA_OCULTA]

# Carrega dados de documentos
doc_encoded_restaurado = restaurar_doc_encoded_de_pickle(PASTA_DOCUMENTOS)
xb = doc_encoded_restaurado[TIPO_CAMADA_OCULTA]
id_list = [item.replace('JURISPRUDENCIA-SELECIONADA-LEGADA-', '') for item in doc_encoded_restaurado['key']]
id_list = [item.replace('JURISPRUDENCIA-SELECIONADA-', '') for item in id_list]
id_list = list(map(int, id_list))

## 2. Construindo um índice e adicionando os vetores a ele

In [3]:
import faiss                   # make faiss available

#Cria índice com mapeamento de ID (IndexIDMap)
index = faiss.IndexIDMap(faiss.IndexFlatL2(len(xb[0])))

# Verifica se o índice já está treinado
print(index.is_trained)

# Adiciona os vetores ao índice do banco de dados
index.add_with_ids(xb, id_list)

# Verifica o número de vetores indexados
print(index.ntotal)

True
16045


## 3. Buscando

In [4]:
# k é o número de vetores que serão retornados na busca
k = 50

# teste de sanidade do banco
#D, I = index.search(xb[:5], k) # sanity check
#print(I)
#print(D)

# busca propriamente dita
D, I = index.search(xq, k)     # actual search

# Gravando lote em um arquivo .pickle
caminho_arquivo = f'{PASTA_RESULTADO_CADERNO}{CAMINHO_MODELO}_{TIPO_CAMADA_OCULTA}_resultado_query.pickle'
with open(caminho_arquivo, 'wb') as arquivo_pickle:
    pickle.dump(I, arquivo_pickle)

# I é uma matriz inteira de tamanho nq x k, onde a linha i contém os IDs dos k vizinhos do vetor de consulta i,
# ordenados por distância crescente

# D é uma matriz de ponto flutuante nq x k com as distâncias quadráticas correspondentes